In [2]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install -qU langchain-teddynote


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [10]:
!pip install langchain langchain-openai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [11]:
!pip install langchain_community

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 18.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install faiss-gpu

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.5 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install langchain.document

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement langchain.document (from versions: none)
ERROR: No matching distribution found for langchain.document

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [10]:
!pip install jq

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 15.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [6]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [7]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력
logging.langsmith("ChatBot_Project")

LangSmith 추적을 시작합니다.
[프로젝트명]
ChatBot_Project


In [8]:
import os

os.environ['LANGCHAIN_PROJECT'] = 'ChatBot_Project'

In [35]:
# 필요한 모듈 import
import os
import faiss
from tqdm import tqdm
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer  # Hugging Face 모델 사용
import torch
from langchain_openai import ChatOpenAI  # ChatOpenAI import
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser
from langchain.document_loaders import JSONLoader
from langchain_community.docstore.in_memory import InMemoryDocstore

# 벡터 저장 및 불러오기 함수
def save_faiss_index(index, filename):
    faiss.write_index(index, filename)

def load_faiss_index(filename):
    return faiss.read_index(filename)

# 각 JSON 파일 경로
file_paths = {
    "국정감사": {
        "path": '국정감사.json',
        "description": "This file contains information and records related to the National Audit."
    },
    "본회의": {
        "path": '본회의.json',
        "description": "This file contains details and proceedings from the Plenary Session."
    },
    "소위원회": {
        "path": '소위원회.json',
        "description": "This file includes records and discussions from the Subcommittee meetings."
    },
    "예산결산특별위원회": {
        "path": '예산결산특별위원회.json',
        "description": "This file contains information regarding the Special Committee on Budget and Settlement."
    },
    "특별위원회": {
        "path": '특별위원회.json',
        "description": "This file contains records and details from the Special Committee meetings."
    }
}

# 영어로 매핑된 위원회 이름
committee_name_mapping = {
    "국정감사": "National_Audit",
    "본회의": "Plenary_Session",
    "소위원회": "Subcommittee",
    "예산결산특별위원회": "Budget_Special_Committee",
    "특별위원회": "Special_Committee"
}

# JSONLoader의 jq_schema 설정
jq_schema = """
.[] | {
    "filename": .filename,
    "original": .original,
    "id": .id,
    "date": .date,
    "conference_number": .conference_number,
    "question_number": .question_number,
    "meeting_name": .meeting_name,
    "generation_number": .generation_number,
    "committee_name": .committee_name,
    "meeting_number": .meeting_number,
    "session_number": .session_number,
    "agenda": .agenda,
    "law": .law,
    "qna_type": .qna_type,
    "context": .context,
    "context_learn": .context_learn,
    "context_summary": {
        "summary_q": .context_summary.summary_q,
        "summary_a": .context_summary.summary_a
    },
    "questioner_name": .questioner_name,
    "questioner_ID": .questioner_ID,
    "questioner_ISNI": .questioner_ISNI,
    "questioner_affiliation": .questioner_affiliation,
    "questioner_position": .questioner_position,
    "question": {
        "tag": .question.tag,
        "comment": .question.comment,
        "keyword": .question.keyword
    },
    "answerer_name": .answerer_name,
    "answerer_ID": .answerer_ID,
    "answerer_ISNI": .answerer_ISNI,
    "answerer_affiliation": .answerer_affiliation,
    "answerer_position": .answerer.position,
    "answer": {
        "tag": .answer.tag,
        "comment": .answer.comment,
        "keyword": .answer.keyword
    }
}
"""

# 각 파일에 대해 JSONLoader, 분할기, 벡터 저장소 및 도구 생성
retriever_tools = {}
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # GPU 사용 여부 확인
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens', device=device)  # 허깅페이스 모델 로드

# 임베딩 차원 수 설정
embed_dimension = 768  # DistilBERT의 임베딩 차원 수

# 임베딩 함수 정의
def embedding_function(texts):
    return model.encode(texts, convert_to_tensor=True).cpu().numpy()

for committee_name, file_info in tqdm(file_paths.items()):
    english_name = committee_name_mapping[committee_name]
    file_path = file_info["path"]
    
    # JSONLoader를 사용하여 파일 로드
    loader = JSONLoader(file_path, jq_schema=jq_schema, text_content=False)
    documents = loader.load()

    # 텍스트 분할기를 사용하여 문서를 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(documents)

    # 저장할 벡터 저장소 파일 경로
    vectorstore_file = f'{english_name}_faiss_index.index'

    # 문서 내용을 리스트로 변환
    document_contents = [doc.page_content for doc in split_docs]

    # 인덱스와 ID 매핑 생성
    index_to_docstore_id = {i: i for i in range(len(document_contents))}

    # 문서 내용을 InMemoryDocstore로 변환
    docstore = InMemoryDocstore({i: doc for i, doc in enumerate(split_docs)})

    # VectorStore가 이미 존재하는지 확인
    if os.path.exists(vectorstore_file):
        index = load_faiss_index(vectorstore_file)
        vector = FAISS(index=index, embedding_function=embedding_function, docstore=docstore, index_to_docstore_id=index_to_docstore_id)
    else:
        # VectorStore를 생성
        embeddings = embedding_function(document_contents)  # 문서의 텍스트를 임베딩
        index = faiss.IndexFlatL2(embed_dimension)
        index.add(embeddings)  # FAISS에 추가
        save_faiss_index(index, vectorstore_file)
        vector = FAISS(index=index, embedding_function=embedding_function, docstore=docstore, index_to_docstore_id=index_to_docstore_id)

    # Retriever를 생성
    retriever = vector.as_retriever()

    # retriever_tool 정의
    retriever_tool = create_retriever_tool(
        retriever,
        name=f"{english_name}_search",
        description=f"use this tool to search information from the {committee_name} JSON document."
                    f"the {english_name} is related with meeting_name as JSON document's key."
                    f"{file_info['description']}"
    )

    # 도구를 딕셔너리에 저장
    retriever_tools[committee_name] = retriever_tool

### 1-3. tools 리스트에 도구 목록을 추가 ###
tools = list(retriever_tools.values())

########## 2. LLM 을 정의 ##########
llm = ChatOpenAI(model="gpt-4o", temperature=0)  # ChatOpenAI 모델 정의



100%|██████████| 5/5 [03:22<00:00, 40.55s/it]


In [51]:
########## 3. Prompt 를 정의 ##########
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You should help the people who are asking you questions as a helpful assistant."
            "Make sure to use the `*_search` tools for searching information from the JSON document."
            "'*_search' tools are for the National Audit, the Plenary Session, the Subcommittee meetings, the Special Committee on Budget and Settlement, and the Special Committee meetings."
            "All the JSON documents' contents are National Assembly Proceedings."
            "You must strictly refer to the information contained in the JSON files."
            "You are explaining information from JSON documents' keys."
            "Date is the date when meeting_name is open. Generation number is what number National Assembly it is."
            "Context is conversation as raw data. Context_learn is good for learning about the conversation."
            "Summary_q is the question's summary and summary_a is the answer's summary."
            "When you search for answers to questions, you should primarily check keys on JSON documents."
            "The keys are date, meeting_name, generation_number, committee_name, agenda, context_learn, context_summary, questioner_name, answerer_name, question, and answer."
            "If a person asking a question has not defined the scope of that question, all relevant JSON files should be searched."
            "If you receive a question regarding a person's name, you should search with questioner_name or answerer_name."
            "You must only provide answers based on the information in the JSON files."
            "Do not provide answers that are not derived from the data."
        ),
        # (
        #     "system",
        #     "When you respond to inquiries about a specific meeting., please follow this format:\n"
        #     "- 날짜: The date when the meeting occurred.\n"
        #     "- 회의명: The name of the meeting.\n"
        #     "- 국회대: The number of the National Assembly.\n"
        #     "- 위원회명: The name of the committee involved.\n"
        #     "- 안건: The agenda discussed.\n"
        #     "- 질문자: The person's name who asked the question.\n"
        #     "- 응답자: The person's name who answered.\n"
        #     "- 질문 내용: The question asked in detail.\n"
        #     "- 응답 내용: The response given in detail.\n\n"
        #     "- 요약: A summary of the question and the answer.\n"
        #     "If the user's question lacks scope, search all relevant JSON files."
        #     "Please fill out all data for the format."
        # ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의 ##########
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의 ##########
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리를 추가 ##########
store = {}

def get_session_history(session_ids):
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의 ##########
agent_stream_parser = AgentStreamParser()

In [54]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "그럼 가짜정보 아니야?"},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: National_Audit_search
query: 최근 회의 날짜
Log: 
Invoking: `National_Audit_search` with `{'query': '최근 회의 날짜'}`



[도구 호출]
Tool: Plenary_Session_search
query: 최근 회의 날짜
Log: 
Invoking: `Plenary_Session_search` with `{'query': '최근 회의 날짜'}`



[도구 호출]
Tool: Subcommittee_search
query: 최근 회의 날짜
Log: 
Invoking: `Subcommittee_search` with `{'query': '최근 회의 날짜'}`



[도구 호출]
Tool: Budget_Special_Committee_search
query: 최근 회의 날짜
Log: 
Invoking: `Budget_Special_Committee_search` with `{'query': '최근 회의 날짜'}`



[도구 호출]
Tool: Special_Committee_search
query: 최근 회의 날짜
Log: 
Invoking: `Special_Committee_search` with `{'query': '최근 회의 날짜'}`



[관찰 내용]
Observation: "answerer_ID": " ", "answerer_ISNI": " ", "answerer_affiliation": "\uad6d\ubc29\ubd80", "answerer_position": null, "answer": {"tag": "330", "comment": "\uc608, \uc81c\uac00 \uc774 \ub0b4\uc6a9\uc744 \uc811\ud558\uace0 \ub9e4\uc6b0 \ubd80\uc801\uc808\ud55c \uc6b0\ub9ac \ud589\uc815 \uc808\ucc28\ub97c \ud588\ub2e4 \uc774\ub807\uac8c \u

In [73]:
import pandas as pd
import numpy as np
import json

In [81]:
# JSON 파일 읽기
with open('국정감사.json', 'r', encoding='utf-8') as f:
    data_list = json.load(f)

# 데이터프레임 생성
df = pd.DataFrame([
    {
        'filename': item['filename'],
        'original': item['original'],
        'id': item['id'],
        'date': item['date'],
        'conference_number': item['conference_number'],
        'question_number': item['question_number'],
        'meeting_name': item['meeting_name'],
        'generation_number': item['generation_number'],
        'committee_name': item['committee_name'],
        'meeting_number': item['meeting_number'],
        'session_number': item['session_number'],
        'agenda': item['agenda'],
        'law': item['law'],
        'qna_type': item['qna_type'],
        'context': item['context'],
        'context_learn': item['context_learn'],
        'context_summary_q': item['context_summary']['summary_q'],
        'context_summary_a': item['context_summary']['summary_a'],
        'questioner_name': item['questioner_name'],
        "questioner_ID": item['questioner_ID'],
        "questioner_ISNI": item['questioner_ISNI'],
        "questioner_affiliation":item['questioner_affiliation'],
        'questioner_position': item['questioner_position'],
        'question': item['question']['comment'],
        'question_tag': item['question']['tag'],
        'question_keyword': item['question']['keyword'],
        'answerer_name': item['answerer_name'],
        "answerer_ID": item['answerer_ID'],
        "answerer_ISNI": item['answerer_ISNI'],
        "answerer_affiliation":item['answerer_affiliation'],
        'answerer_position': item['answerer_position'],
        'answer': item['answer']['comment'],
        'answer_tag': item['answer']['tag'],
        'answer_keyword': item['answer']['keyword']
    } for item in data_list
])

df

,filename,original,id,date,conference_number,question_number,meeting_name,generation_number,committee_name,meeting_number,...,question_tag,question_keyword,answerer_name,answerer_ID,answerer_ISNI,answerer_affiliation,answerer_position,answer,answer_tag,answer_keyword
0,SRC_16대_2000_2000년10월20일_국정감사_교육위원회_0001(03004...,http://likms.assembly.go.kr/record/mhs-10-040-...,030043,2000年10月20日(金),030043,0001,국정감사,16,교육위원회,2000,...,535,"비평준화지역, 평준화지역, 향상",조성윤,,,경기도,교육감,"평준화문제는 지금 성남ㆍ분당지역, 고양ㆍ일산지역, 안양지역, 부천지역 그 외에 이삼...",536,"교육청, 평준화문제, 성남ㆍ분당지역, 고양ㆍ일산지역, 한국교육개발원"
1,SRC_16대_2000_2000년10월23일_국정감사_행정자치위원회-제1반_0001...,http://likms.assembly.go.kr/record/mhs-10-040-...,028685,2000年10月23日(月),028685,0001,국정감사,16,행정자치위원회-제1반,2000,...,124,"선거구별, 단속실적, 형사입건대상, 선거구별 형사입건",배희선,,,전라남도지방경찰청,청장,"위원님, 즉답 드리겠습니다. 제가 아까 한번 말씀드렸습니다마는 금년에 오고 나서...",125,"전남경찰청, 구조조정, 경찰, 부족"
2,SRC_16대_2000_2000년10월30일_국정감사_법제사법위원회_0001(028...,http://likms.assembly.go.kr/record/mhs-10-040-...,028928,2000年10月30日(月),028928,0001,국정감사,16,법제사법위원회,2000,...,160,"부산지검, 검사장, 지역사회, 유착, 다대ㆍ만덕사건, 국정감사, 감사원 감사",송광수,,,부산지방검찰청,검사장,인사가 된 것이 금년 2월인데 그때는 이영복이가 이미 도주했고 나머지 사람들에 대해...,161,"승진, 인사, 이영복, 도주, 수사검사"
3,SRC_16대_2000_2000년10월30일_국정감사_산업자원위원회_0001(028...,http://likms.assembly.go.kr/record/mhs-10-040-...,028961,2000年10月30日(月),028961,0001,국정감사,16,산업자원위원회,2000,...,239,"경남, 청소년들, 부탄가스, 흡입, 지역구, 진해",김영대,,,한국가스안전공사,사장,고미제 첨가가 98년도 7월1일 이후로 의무화하도록 되어 있었습니다. 그런데 그 내...,240,"고미제, 첨가, 혼합, 부탄가스"
4,SRC_16대_2000_2000년10월31일_국정감사_통일외교통상위원회-아주반_00...,http://likms.assembly.go.kr/record/mhs-10-040-...,029591,2000年10月31日(火),029591,0001,국정감사,16,통일외교통상위원회-아주반,2000,...,67,"대상, 샘플, 노벨상, 수상, 남북정상회담, 대통령의, 대북",장세창,,,주중화인민공화국대한민국대사관,공사(문화홍보),중국에서 여론조사 형식을 빌어서 조사하는 것에는 애로가 있습니다. 저희들이 96년...,68,"조사, 중국, 여론조사, 형식, 반응, 언론"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1787,SRC_21대_2022_2022년10월20일_국정감사_외교통일위원회-아주반_0005...,http://likms.assembly.go.kr/record/mhs-10-040-...,052615,2022년10월20일(목),052615,0005,국정감사,21,외교통일위원회-아주반,2022,...,94,"업무보고서, 데, 호주, 금지법, 워홀러들, 경우, 노출, 가능성",홍상우,,,주시드니대한민국총영사관,,"맞는 말씀입니다. 그래서 위원님 말씀 반영해 가지고, 저희가 지금 생활법률 안내 4...",95,"말씀, 위원님, 반영해, 가지, 저희, 생활법률, 4차, 개정판, 준비"
1788,SRC_21대_2022_2022년10월21일_국정감사_국방위원회_0014(05260...,http://likms.assembly.go.kr/record/mhs-10-040-...,052607,2022년10월21일(금),052607,0014,국정감사,21,국방위원회,2022,...,286,"휴대폰, 폐쇄된 공간, 제도 개선, 함정, 해경",이종호,,,해군,참모총장,지금 국방부에 소요 제기를 해서 인사혁신처하고 기재부와 계속 협의 중에 있습니다.,287,"국방부, 인사혁신처, 기재부"
1789,SRC_21대_2022_2022년10월24일_국정감사_국방위원회_0009(05227...,http://likms.assembly.go.kr/record/mhs-10-040-...,052279,2022년10월24일(월),052279,0009,국정감사,21,국방위원회,2022,...,738,,,,,,,"제가 금요일 이사 날짜도 4월 25일로 잘못 답변을 드렸고, 제가 다시 한번 날짜를...",739,
1790,SRC_21대_2022_2022년10월24일_국정감사_법제사법위원회_0011(052...,http://likms.assembly.go.kr/record/mhs-10-040-...,052309,2022년10월24일(월),052309,0011,국정감사,21,법제사법위원회,2022,...,400,"회유, 유동규, 불법정치자금, 수사, 피의자, 신변보호, 위험",한동훈,,,법무부,장관,제가 구체적인 사안에 대해서 말씀드린 것은 그렇습니다만 일반론을 말씀드리자면 이런 ...,401,"수사, 구체적, 사안, 일반론, 대형, 부정․비리, 객관적, 증거"


In [82]:
df.columns

Index(['filename', 'original', 'id', 'date', 'conference_number',
       'question_number', 'meeting_name', 'generation_number',
       'committee_name', 'meeting_number', 'session_number', 'agenda', 'law',
       'qna_type', 'context', 'context_learn', 'context_summary_q',
       'context_summary_a', 'questioner_name', 'questioner_ID',
       'questioner_ISNI', 'questioner_affiliation', 'questioner_position',
       'question', 'question_tag', 'question_keyword', 'answerer_name',
       'answerer_ID', 'answerer_ISNI', 'answerer_affiliation',
       'answerer_position', 'answer', 'answer_tag', 'answer_keyword'],
      dtype='object')

In [83]:
# 공백 문자열을 NaN으로 변경
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

# 결과 확인
df

,filename,original,id,date,conference_number,question_number,meeting_name,generation_number,committee_name,meeting_number,...,question_tag,question_keyword,answerer_name,answerer_ID,answerer_ISNI,answerer_affiliation,answerer_position,answer,answer_tag,answer_keyword
0,SRC_16대_2000_2000년10월20일_국정감사_교육위원회_0001(03004...,http://likms.assembly.go.kr/record/mhs-10-040-...,030043,2000年10月20日(金),030043,0001,국정감사,16,교육위원회,2000,...,535,"비평준화지역, 평준화지역, 향상",조성윤,NaN,NaN,경기도,교육감,"평준화문제는 지금 성남ㆍ분당지역, 고양ㆍ일산지역, 안양지역, 부천지역 그 외에 이삼...",536,"교육청, 평준화문제, 성남ㆍ분당지역, 고양ㆍ일산지역, 한국교육개발원"
1,SRC_16대_2000_2000년10월23일_국정감사_행정자치위원회-제1반_0001...,http://likms.assembly.go.kr/record/mhs-10-040-...,028685,2000年10月23日(月),028685,0001,국정감사,16,행정자치위원회-제1반,2000,...,124,"선거구별, 단속실적, 형사입건대상, 선거구별 형사입건",배희선,NaN,NaN,전라남도지방경찰청,청장,"위원님, 즉답 드리겠습니다. 제가 아까 한번 말씀드렸습니다마는 금년에 오고 나서...",125,"전남경찰청, 구조조정, 경찰, 부족"
2,SRC_16대_2000_2000년10월30일_국정감사_법제사법위원회_0001(028...,http://likms.assembly.go.kr/record/mhs-10-040-...,028928,2000年10月30日(月),028928,0001,국정감사,16,법제사법위원회,2000,...,160,"부산지검, 검사장, 지역사회, 유착, 다대ㆍ만덕사건, 국정감사, 감사원 감사",송광수,NaN,NaN,부산지방검찰청,검사장,인사가 된 것이 금년 2월인데 그때는 이영복이가 이미 도주했고 나머지 사람들에 대해...,161,"승진, 인사, 이영복, 도주, 수사검사"
3,SRC_16대_2000_2000년10월30일_국정감사_산업자원위원회_0001(028...,http://likms.assembly.go.kr/record/mhs-10-040-...,028961,2000年10月30日(月),028961,0001,국정감사,16,산업자원위원회,2000,...,239,"경남, 청소년들, 부탄가스, 흡입, 지역구, 진해",김영대,NaN,NaN,한국가스안전공사,사장,고미제 첨가가 98년도 7월1일 이후로 의무화하도록 되어 있었습니다. 그런데 그 내...,240,"고미제, 첨가, 혼합, 부탄가스"
4,SRC_16대_2000_2000년10월31일_국정감사_통일외교통상위원회-아주반_00...,http://likms.assembly.go.kr/record/mhs-10-040-...,029591,2000年10月31日(火),029591,0001,국정감사,16,통일외교통상위원회-아주반,2000,...,67,"대상, 샘플, 노벨상, 수상, 남북정상회담, 대통령의, 대북",장세창,NaN,NaN,주중화인민공화국대한민국대사관,공사(문화홍보),중국에서 여론조사 형식을 빌어서 조사하는 것에는 애로가 있습니다. 저희들이 96년...,68,"조사, 중국, 여론조사, 형식, 반응, 언론"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1787,SRC_21대_2022_2022년10월20일_국정감사_외교통일위원회-아주반_0005...,http://likms.assembly.go.kr/record/mhs-10-040-...,052615,2022년10월20일(목),052615,0005,국정감사,21,외교통일위원회-아주반,2022,...,94,"업무보고서, 데, 호주, 금지법, 워홀러들, 경우, 노출, 가능성",홍상우,NaN,NaN,주시드니대한민국총영사관,NaN,"맞는 말씀입니다. 그래서 위원님 말씀 반영해 가지고, 저희가 지금 생활법률 안내 4...",95,"말씀, 위원님, 반영해, 가지, 저희, 생활법률, 4차, 개정판, 준비"
1788,SRC_21대_2022_2022년10월21일_국정감사_국방위원회_0014(05260...,http://likms.assembly.go.kr/record/mhs-10-040-...,052607,2022년10월21일(금),052607,0014,국정감사,21,국방위원회,2022,...,286,"휴대폰, 폐쇄된 공간, 제도 개선, 함정, 해경",이종호,NaN,NaN,해군,참모총장,지금 국방부에 소요 제기를 해서 인사혁신처하고 기재부와 계속 협의 중에 있습니다.,287,"국방부, 인사혁신처, 기재부"
1789,SRC_21대_2022_2022년10월24일_국정감사_국방위원회_0009(05227...,http://likms.assembly.go.kr/record/mhs-10-040-...,052279,2022년10월24일(월),052279,0009,국정감사,21,국방위원회,2022,...,738,NaN,NaN,NaN,NaN,NaN,NaN,"제가 금요일 이사 날짜도 4월 25일로 잘못 답변을 드렸고, 제가 다시 한번 날짜를...",739,NaN
1790,SRC_21대_2022_2022년10월24일_국정감사_법제사법위원회_0011(052...,http://likms.assembly.go.kr/record/mhs-10-040-...,052309,2022년10월24일(월),052309,0011,국정감사,21,법제사법위원회,2022,...,400,"회유, 유동규, 불법정치자금, 수사, 피의자, 신변보호, 위험",한동훈,NaN,NaN,법무부,장관,제가 구체적인 사안에 대해서 말씀드린 것은 그렇습니다만 일반론을 말씀드리자면 이런 ...,401,"수사, 구체적, 사안, 일반론, 대형, 부정․비리, 객관적, 증거"


In [84]:
df.isna().sum()

filename                     0
original                     0
id                           0
date                         0
conference_number            0
question_number              0
meeting_name                 0
generation_number            0
committee_name               0
meeting_number               0
session_number               0
agenda                       0
law                       1697
qna_type                     0
context                      0
context_learn                0
context_summary_q            0
context_summary_a            0
questioner_name              1
questioner_ID                7
questioner_ISNI              7
questioner_affiliation    1780
questioner_position          4
question                     0
question_tag                 0
question_keyword            14
answerer_name                1
answerer_ID               1785
answerer_ISNI             1785
answerer_affiliation       200
answerer_position            2
answer                       0
answer_t